In [1]:
#checking tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#importing libraries 

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
# Cross Validation dataset links

#CV-1
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 1/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 1/Validation/"

#CV-2
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 2/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 2/Validation/"

#CV-3
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 3/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 3/Validation/"

#CV-4
train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 4/Training/"
val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 4/Validation/"

#CV-5
#train_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 5/Training/"
#val_dir="/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG_version_2/DA_version_3/Model_Training/K-Fold Cross Validation/Fold 5/Validation/"

In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 31206 files belonging to 5 classes.


2023-01-06 20:47:51.943833: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 20:47:51.990943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 20:47:51.991283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 20:47:51.992358: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 7787 files belonging to 5 classes.


In [5]:
#Normalizing the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
#import tensorflow.python.keras.applications.VGG16
base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [16]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [17]:
#defining callback 
class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [18]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-01-06 20:48:20.240559: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-01-06 20:48:20.665083: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


244/244 [==============================] - ETA: 0s - loss: 1.1921 - accuracy: 0.4979 - sensitivity_at_specificity_1: 0.8917 - specificity_at_sensitivity_1: 0.8845 - recall_1: 0.2856 - precision_1: 0.6954
Epoch 1: val_accuracy improved from -inf to 0.63439, saving model to ECG_Spectrogram_Model.h5
244/244 [==============================] - 203s 788ms/step - loss: 1.1921 - accuracy: 0.4979 - sensitivity_at_specificity_1: 0.8917 - specificity_at_sensitivity_1: 0.8845 - recall_1: 0.2856 - precision_1: 0.6954 - val_loss: 0.8951 - val_accuracy: 0.6344 - val_sensitivity_at_specificity_1: 0.9756 - val_specificity_at_sensitivity_1: 0.9561 - val_recall_1: 0.5047 - val_precision_1: 0.7383
Epoch 2/50
244/244 [==============================] - ETA: 0s - loss: 0.8968 - accuracy: 0.6437 - sensitivity_at_specificity_1: 0.9725 - specificity_at_sensitivity_1: 0.9585 - recall_1: 0.5203 - precision_1: 0.7408
Epoch 2: val_accuracy did not improve from 0.63439
244/244 [==============================] - 182s

244/244 [==============================] - ETA: 0s - loss: 0.5590 - accuracy: 0.7901 - sensitivity_at_specificity_1: 0.9928 - specificity_at_sensitivity_1: 0.9925 - recall_1: 0.7451 - precision_1: 0.8314
Epoch 13: val_accuracy improved from 0.71658 to 0.72184, saving model to ECG_Spectrogram_Model.h5
244/244 [==============================] - 176s 722ms/step - loss: 0.5590 - accuracy: 0.7901 - sensitivity_at_specificity_1: 0.9928 - specificity_at_sensitivity_1: 0.9925 - recall_1: 0.7451 - precision_1: 0.8314 - val_loss: 0.8141 - val_accuracy: 0.7218 - val_sensitivity_at_specificity_1: 0.9735 - val_specificity_at_sensitivity_1: 0.9789 - val_recall_1: 0.6888 - val_precision_1: 0.7466
Epoch 14/50
244/244 [==============================] - ETA: 0s - loss: 0.5608 - accuracy: 0.7895 - sensitivity_at_specificity_1: 0.9956 - specificity_at_sensitivity_1: 0.9919 - recall_1: 0.7447 - precision_1: 0.8309
Epoch 14: val_accuracy did not improve from 0.72184
244/244 [==============================] 

244/244 [==============================] - 177s 725ms/step - loss: 0.4362 - accuracy: 0.8369 - sensitivity_at_specificity_1: 0.9984 - specificity_at_sensitivity_1: 0.9969 - recall_1: 0.8079 - precision_1: 0.8654 - val_loss: 1.0377 - val_accuracy: 0.7009 - val_sensitivity_at_specificity_1: 0.9441 - val_specificity_at_sensitivity_1: 0.9710 - val_recall_1: 0.6763 - val_precision_1: 0.7245
Epoch 26/50
244/244 [==============================] - ETA: 0s - loss: 0.4370 - accuracy: 0.8364 - sensitivity_at_specificity_1: 0.9983 - specificity_at_sensitivity_1: 0.9965 - recall_1: 0.8059 - precision_1: 0.8644
Epoch 26: val_accuracy did not improve from 0.72557
244/244 [==============================] - 177s 724ms/step - loss: 0.4370 - accuracy: 0.8364 - sensitivity_at_specificity_1: 0.9983 - specificity_at_sensitivity_1: 0.9965 - recall_1: 0.8059 - precision_1: 0.8644 - val_loss: 1.1196 - val_accuracy: 0.6834 - val_sensitivity_at_specificity_1: 0.9507 - val_specificity_at_sensitivity_1: 0.9684 - v

Epoch 38/50
244/244 [==============================] - ETA: 0s - loss: 0.3616 - accuracy: 0.8641 - sensitivity_at_specificity_1: 0.9988 - specificity_at_sensitivity_1: 0.9985 - recall_1: 0.8431 - precision_1: 0.8859
Epoch 38: val_accuracy did not improve from 0.72557
244/244 [==============================] - 177s 725ms/step - loss: 0.3616 - accuracy: 0.8641 - sensitivity_at_specificity_1: 0.9988 - specificity_at_sensitivity_1: 0.9985 - recall_1: 0.8431 - precision_1: 0.8859 - val_loss: 1.2936 - val_accuracy: 0.6795 - val_sensitivity_at_specificity_1: 0.9325 - val_specificity_at_sensitivity_1: 0.9649 - val_recall_1: 0.6575 - val_precision_1: 0.6976
Epoch 39/50
244/244 [==============================] - ETA: 0s - loss: 0.3283 - accuracy: 0.8769 - sensitivity_at_specificity_1: 0.9990 - specificity_at_sensitivity_1: 0.9991 - recall_1: 0.8580 - precision_1: 0.8938
Epoch 39: val_accuracy did not improve from 0.72557
244/244 [==============================] - 177s 724ms/step - loss: 0.3283 -

In [20]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity_1']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity_1']
Validation_Recall=history.history['val_recall_1']
Validation_Precision=history.history['val_precision_1']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.8946036100387573
Validation Accuracy:  0.7255682349205017
Validation Specificity:  0.9816360473632812
Validation Sensitivity:  0.9818928837776184
Validation Recall:  0.6919224262237549
Validation Precision:  0.7701149582862854
Validation Loss:  0.7729121446609497
